# Libs

In [1]:
import numpy as np
import pandas as pd

from cloudmanufacturing.data import read_fatahi_dataset
from cloudmanufacturing.solvers.naive_solver import NaiveSolver

/Users/fomindmitry/miniconda3/envs/graph/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading data

In [2]:
dataset = read_fatahi_dataset("../data/fatahi.xlsx")

  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:04<00:00,  3.73it/s]


# Solving problem

In [3]:
naive_solver = NaiveSolver(dataset)

In [4]:
info_problem = naive_solver.solve_problem(0)
info_problem

{'path': {'suboperation_0': array([0., 3., 0., 0., 3.]),
  'suboperation_1': array([3., 3., 0., 0.]),
  'suboperation_2': array([8., 3., 3., 3., 6., 6.]),
  'suboperation_3': array([6., 6., 3., 3., 0., 3., 3.]),
  'suboperation_4': array([3., 3., 3., 0., 0., 3.])},
 'cost': 6141.712220372922}

In [5]:
info = naive_solver.solve_all()
print("total_cost:", naive_solver.total_cost)
info

total_cost: 224420.56560195843


{'problem_0': {'path': {'suboperation_0': array([0., 3., 0., 0., 3.]),
   'suboperation_1': array([3., 3., 0., 0.]),
   'suboperation_2': array([8., 3., 3., 3., 6., 6.]),
   'suboperation_3': array([6., 6., 3., 3., 0., 3., 3.]),
   'suboperation_4': array([3., 3., 3., 0., 0., 3.])},
  'cost': 6141.712220372922},
 'problem_1': {'path': {'suboperation_0': array([9., 9., 9., 5., 5., 7., 7., 6., 6.]),
   'suboperation_1': array([8., 3., 0., 3., 0., 0., 3., 6., 6.]),
   'suboperation_2': array([6., 3., 0., 3., 6.]),
   'suboperation_3': array([8., 3., 3., 3., 0., 0., 6.]),
   'suboperation_4': array([6., 5., 5., 5., 6., 6.])},
  'cost': 10863.98114111436},
 'problem_2': {'path': {'suboperation_0': array([9., 9., 9., 9., 5., 5., 7., 7., 6., 6.]),
   'suboperation_1': array([8., 3., 0., 3., 3., 3., 3.]),
   'suboperation_2': array([3., 3., 0., 3., 3., 0., 0., 3., 6., 6.]),
   'suboperation_3': array([8., 3., 0., 3., 3., 0., 3., 6.]),
   'suboperation_4': array([6., 3., 3., 0., 0., 6.])},
  'c